In [1]:
from pathlib import Path

def find_project_root(start: Path) -> Path:
    p = start.resolve()
    for parent in [p] + list(p.parents):
        if (parent / "data").exists():
            return parent
    return p

ROOT = find_project_root(Path.cwd())

ROOT

WindowsPath('C:/Users/HiWi/Desktop/Terril/01_nextcloud/Germany/DATA SCIENCE/Semesters/05/02 Sustainability in aviation/03 contrail-mvp')

## Step 1 — Load segments + ERA5 parquet

In [2]:

import pandas as pd

SEG_PATH = ROOT / "data/processed/flight_data/states_europe/2025-01-13_2025-01-14/opensky_segments_europe_winter_day.parquet"
# if your file is named slightly differently, adjust the name above

ERA5_PQ = ROOT / "data/processed/weather_ERA5/states_europe/2025-01-13_2025-01-14/era5_pl_europe_20250113_T_q.parquet"

segments = pd.read_parquet(SEG_PATH)
era5 = pd.read_parquet(ERA5_PQ)

print("segments:", segments.shape)
print("era5:", era5.shape)
segments.head()


segments: (1336164, 7)
era5: (3593880, 6)


,flight_id,mid_time,mid_lat,mid_lon,mid_alt_m,dt_s,dist_km
0,008ba5_OPM007_1.0,1736754704,50.464413,-0.393166,8164.83,31,7.231921
1,008ba5_OPM007_1.0,1736754750,50.372501,-0.339611,8667.75,60,14.612554
2,008ba5_OPM007_1.0,1736754810,50.245132,-0.275096,9277.35,60,15.158267
3,008ba5_OPM007_1.0,1736754870,50.112259,-0.211920,9776.46,60,15.735089
4,008ba5_OPM007_1.0,1736754930,49.973442,-0.155471,10264.14,60,16.179957


## Step 2 - Prepare keys for joining

### 2a. Segment time → nearest ERA5 hour

In [3]:
import numpy as np

segments["dt_utc"] = pd.to_datetime(segments["mid_time"], unit="s", utc=True)
segments["era5_time"] = segments["dt_utc"].dt.round("H")


C:\Users\HiWi\AppData\Local\Temp\ipykernel_25024\273478478.py:4: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  segments["era5_time"] = segments["dt_utc"].dt.round("H")


### 2b Segment altitude → nearest pressure level (hPa)

We’ll map altitude to pressure using the same standard-atmosphere approximation and then snap to nearest ERA5 level available.

In [4]:
def alt_m_to_pressure_pa(h_m: np.ndarray) -> np.ndarray:
    h = np.clip(h_m, 0, 20000)
    p0 = 101325.0
    T0 = 288.15
    L = 0.0065
    g = 9.80665
    R = 287.05
    return p0 * (1 - L*h/T0) ** (g/(R*L))

segments["p_hpa_est"] = alt_m_to_pressure_pa(segments["mid_alt_m"].to_numpy()) / 100.0

plevs = np.sort(era5["plev_hpa"].unique())
def nearest_plev(p):
    return plevs[np.argmin(np.abs(plevs - p))]

segments["plev_hpa"] = segments["p_hpa_est"].apply(nearest_plev).astype("int16")


### 2c) Segment lat/lon → nearest ERA5 grid

ERA5 grid is regular. Instead of expensive nearest-neighbor search, we round to nearest grid.

First, detect grid spacing from the ERA5 table (usually 0.25°).

In [5]:
lat_vals = np.sort(era5["lat"].unique())
lon_vals = np.sort(era5["lon"].unique())

# Estimate grid step (most common is 0.25)
lat_step = float(np.round(np.median(np.diff(lat_vals)), 6))
lon_step = float(np.round(np.median(np.diff(lon_vals)), 6))

print("Detected grid step:", lat_step, lon_step)

def snap_to_grid(x, grid_min, step):
    return np.round((x - grid_min)/step)*step + grid_min

segments["lat_g"] = snap_to_grid(segments["mid_lat"].to_numpy(), lat_vals.min(), lat_step).astype("float32")
segments["lon_g"] = snap_to_grid(segments["mid_lon"].to_numpy(), lon_vals.min(), lon_step).astype("float32")


Detected grid step: 0.25 0.25


## Step 3 — Join segments with ERA5

In [7]:

# segments already has era5_time, keep it UTC-aware
segments["era5_time"] = pd.to_datetime(segments["era5_time"], utc=True)

# ERA5 time column: convert to UTC-aware
era5["time"] = pd.to_datetime(era5["time"], utc=True)

# rebuild keyed table
era5_keyed = era5.rename(columns={"time": "era5_time", "lat": "lat_g", "lon": "lon_g"})

merged = segments.merge(
    era5_keyed,
    on=["era5_time", "plev_hpa", "lat_g", "lon_g"],
    how="left",
    validate="many_to_one",
)

print("Merged:", merged.shape)
print("Missing met rows:", merged["T_K"].isna().mean())
merged.head()


Merged: (1336164, 15)
Missing met rows: 0.006271685212294299


,flight_id,mid_time,mid_lat,mid_lon,mid_alt_m,dt_s,dist_km,dt_utc,era5_time,p_hpa_est,plev_hpa,lat_g,lon_g,T_K,q_kgkg
0,008ba5_OPM007_1.0,1736754704,50.464413,-0.393166,8164.83,31,7.231921,2025-01-13 07:51:44+00:00,2025-01-13 08:00:00+00:00,347.586727,350,50.50,-0.50,238.244980,0.000063
1,008ba5_OPM007_1.0,1736754750,50.372501,-0.339611,8667.75,60,14.612554,2025-01-13 07:52:30+00:00,2025-01-13 08:00:00+00:00,322.922639,300,50.25,-0.25,229.575317,0.000032
2,008ba5_OPM007_1.0,1736754810,50.245132,-0.275096,9277.35,60,15.158267,2025-01-13 07:53:30+00:00,2025-01-13 08:00:00+00:00,294.946698,300,50.25,-0.25,229.575317,0.000032
3,008ba5_OPM007_1.0,1736754870,50.112259,-0.211920,9776.46,60,15.735089,2025-01-13 07:54:30+00:00,2025-01-13 08:00:00+00:00,273.532346,250,50.00,-0.25,219.317734,0.000035
4,008ba5_OPM007_1.0,1736754930,49.973442,-0.155471,10264.14,60,16.179957,2025-01-13 07:55:30+00:00,2025-01-13 08:00:00+00:00,253.842040,250,50.00,-0.25,219.317734,0.000035


## Step 4 - Compute RHi and ISSR

In [8]:
import numpy as np

def e_si_pa(T):
    # saturation vapor pressure over ice (Pa), T in Kelvin
    return np.exp(9.550426 - (5723.265 / T) + 3.53068*np.log(T) - 0.00728332*T)

def vapor_pressure_from_q(p_pa, q):
    # q = specific humidity kg/kg
    w = q / np.clip(1 - q, 1e-12, None)  # mixing ratio
    eps = 0.622
    return (w * p_pa) / (eps + w)

p_pa = merged["plev_hpa"].to_numpy(dtype=float) * 100.0
T = merged["T_K"].to_numpy(dtype=float)
q = merged["q_kgkg"].to_numpy(dtype=float)

e = vapor_pressure_from_q(p_pa, q)
esi = e_si_pa(T)

merged["RHi"] = 100.0 * (e / np.clip(esi, 1e-9, None))
merged["ISSR"] = merged["RHi"] > 100.0

merged[["T_K","q_kgkg","plev_hpa","RHi","ISSR"]].head()


,T_K,q_kgkg,plev_hpa,RHi,ISSR
0,238.244980,0.000063,350,15.792362,False
1,229.575317,0.000032,300,18.276783,False
2,229.575317,0.000032,300,18.276783,False
3,219.317734,0.000035,250,57.500223,False
4,219.317734,0.000035,250,57.500223,False


## Step 5 - Score segments, aggregate to flights, pick top 2%

In [9]:
# Simple night proxy (optional)
local_hour = (merged["dt_utc"].dt.hour + merged["mid_lon"] / 15.0) % 24
merged["is_night"] = (local_hour >= 18) | (local_hour < 6)

excess = np.maximum(0.0, (merged["RHi"] - 100.0) / 20.0)
night_w = np.where(merged["is_night"], 1.5, 1.0)

merged["segment_score"] = merged["dist_km"] * excess * night_w * merged["ISSR"].astype(float)

flight_scores = (
    merged.groupby("flight_id", as_index=False)
    .agg(
        flight_score=("segment_score", "sum"),
        dist_in_issr_km=("dist_km", lambda x: float(x[merged.loc[x.index, "ISSR"]].sum())),
        seg_count=("segment_score", "size"),
        issr_seg_count=("ISSR", "sum"),
        mean_RHi=("RHi", "mean"),
    )
    .sort_values("flight_score", ascending=False)
)

n = len(flight_scores)
top_n = max(1, int(round(n * 0.02)))

top2 = flight_scores.head(top_n).copy()

print("Flights:", n)
print("Top 2%:", top_n)
top2.head(10)


Flights: 19986
Top 2%: 400


,flight_id,flight_score,dist_in_issr_km,seg_count,issr_seg_count,mean_RHi
11452,4aca90_NSZ2873_1.0,1438.642334,1885.591834,264,133,100.968493
10911,4952c8_TAP765_1.0,1195.411127,1271.447564,195,92,100.164967
6330,407959_EZY54HC_1.0,1139.190661,1397.404992,162,108,103.948893
17927,502d3e_BTI7MW_1.0,1121.540565,1869.788930,264,137,100.349590
18133,51116f_MBU4WX_1.0,1100.073689,1879.390823,193,141,103.648469
7946,45ab53_JTD262_1.0,1046.059389,1758.059494,221,138,102.168784
14444,4ca891_RYR1BE_1.0,1039.120933,1295.682915,240,92,94.981216
7947,45ab54_JTD298_1.0,1015.505139,1388.747573,218,110,94.294664
7935,45ab4a_JTD372_1.0,989.182319,1633.375201,210,126,105.099995
15427,4cadf7_RYR6EH_1.0,938.712427,914.444548,89,29,79.283618


## Save outputs

In [10]:
OUT_DIR = ROOT / "data/processed/scores/states_europe/2025-01-13_2025-01-14"
OUT_DIR.mkdir(parents=True, exist_ok=True)

flight_scores.to_parquet(OUT_DIR / "flight_scores_rhi_issr.parquet", index=False)
top2.to_parquet(OUT_DIR / "flight_scores_top2pct.parquet", index=False)
merged.to_parquet(OUT_DIR / "segments_with_era5_rhi_issr.parquet", index=False)

print("Saved to:", OUT_DIR)


Saved to: C:\Users\HiWi\Desktop\Terril\01_nextcloud\Germany\DATA SCIENCE\Semesters\05\02 Sustainability in aviation\03 contrail-mvp\data\processed\scores\states_europe\2025-01-13_2025-01-14


In [11]:
import pandas as pd

fs = flight_scores.copy()

n = len(fs)
top_n = max(1, int(round(n*0.02)))
top_share = fs.nlargest(top_n, "flight_score")["flight_score"].sum() / fs["flight_score"].sum()

print("Flights:", n)
print("Top 2% count:", top_n)
print("Score min/median/p95/p99/max:",
      fs["flight_score"].min(),
      fs["flight_score"].median(),
      fs["flight_score"].quantile(0.95),
      fs["flight_score"].quantile(0.99),
      fs["flight_score"].max())
print("Top 2% share of total score:", float(top_share))
print("Zero-score flights:", int((fs["flight_score"] == 0).sum()), f"({(fs['flight_score']==0).mean():.1%})")


Flights: 19986
Top 2% count: 400
Score min/median/p95/p99/max: 0.0 0.0 142.16910295099711 382.04015141202643 1438.6423344313291
Top 2% share of total score: 0.34139988664556864
Zero-score flights: 11737 (58.7%)


In [13]:
import pandas as pd

seg = merged.copy()
total = seg["segment_score"].sum()
night = seg.loc[seg["is_night"], "segment_score"].sum()
print("Night share of score:", float(night/total) if total>0 else 0.0)


Night share of score: 0.5236992619650067


In [ ]:
# --- Visualizations for contrail score project ---
import sys, subprocess
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "pyarrow"])

from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 1) SET THIS to your folder that contains the saved parquet outputs
BASE_DIR = Path(r"data/processed/flight_data/states_europe/2025-01-13_2025-01-14")

# 2) Auto-find files (adjust patterns if your names differ)
fs_candidates = (
    list(BASE_DIR.glob("*flight_scores*issr*.parquet")) +
    list(BASE_DIR.glob("*flight_scores*.parquet"))
)
seg_candidates = (
    list(BASE_DIR.glob("*segments*issr*.parquet")) +
    list(BASE_DIR.glob("*segments*era5*.parquet")) +
    list(BASE_DIR.glob("*segments*.parquet"))
)

print("BASE_DIR:", BASE_DIR)
print("Found flight score files:", [p.name for p in fs_candidates])
print("Found segment files:", [p.name for p in seg_candidates])

if not fs_candidates or not seg_candidates:
    raise FileNotFoundError("Could not find flight_scores / segments parquet. Check BASE_DIR and filenames.")

FS_PATH = fs_candidates[0]
SEG_PATH = seg_candidates[0]

fs = pd.read_parquet(FS_PATH)
seg = pd.read_parquet(SEG_PATH)

print("Loaded fs:", fs.shape, "from", FS_PATH.name)
print("Loaded seg:", seg.shape, "from", SEG_PATH.name)

# ---------- Plot 1: Flight score histogram (linear) ----------
plt.figure()
plt.hist(fs["flight_score"].astype(float), bins=80)
plt.xlabel("Flight score (proxy)")
plt.ylabel("Number of flights")
plt.title("Flight contrail-impact proxy score distribution (linear)")
plt.show()

# ---------- Plot 2: Flight score histogram (log scale via log10(1+score)) ----------
scores = fs["flight_score"].astype(float).to_numpy()
logx = np.log10(1.0 + np.clip(scores, 0, None))

plt.figure()
plt.hist(logx, bins=80)
plt.xlabel("log10(1 + flight_score)")
plt.ylabel("Number of flights")
plt.title("Flight score distribution (log-scaled)")
plt.show()

# ---------- Plot 3: Concentration curve (cumulative share of total score) ----------
fs_sorted = fs.sort_values("flight_score", ascending=False).reset_index(drop=True)
cum = fs_sorted["flight_score"].astype(float).cumsum()
total = float(fs_sorted["flight_score"].astype(float).sum())
cum_share = (cum / total) if total > 0 else cum

x = (fs_sorted.index.to_numpy() + 1) / len(fs_sorted)

plt.figure()
plt.plot(x, cum_share.to_numpy())
plt.xlabel("Fraction of flights (sorted high → low)")
plt.ylabel("Cumulative share of total score")
plt.title("Concentration of contrail-impact proxy score across flights")
plt.show()

# Print your key headline metric
top_n = max(1, int(round(len(fs) * 0.02)))
top_share = fs_sorted.head(top_n)["flight_score"].sum() / fs_sorted["flight_score"].sum()
print(f"Top 2% flights ({top_n}) share of total score: {float(top_share):.2%}")

# ---------- Plot 4: ISSR segment map (sampled) ----------
# requires columns: mid_lat, mid_lon, ISSR
for col in ["mid_lat", "mid_lon", "ISSR"]:
    if col not in seg.columns:
        raise ValueError(f"Segments parquet is missing column '{col}'. Available: {list(seg.columns)}")

issr = seg[seg["ISSR"] == True]
n_sample = min(300_000, len(issr))
if n_sample > 0:
    issr_s = issr.sample(n=n_sample, random_state=1)
    plt.figure()
    plt.scatter(issr_s["mid_lon"], issr_s["mid_lat"], s=1, alpha=0.15)
    plt.xlabel("Longitude")
    plt.ylabel("Latitude")
    plt.title(f"ISSR segment locations (RHi > 100%), sampled n={n_sample:,}")
    plt.show()
else:
    print("No ISSR segments found to plot.")

# ---------- Plot 5: Top-2% flight segment map (sampled) ----------
if "flight_id" not in seg.columns:
    raise ValueError("Segments parquet missing 'flight_id' column.")

top_ids = set(fs_sorted.head(top_n)["flight_id"].astype(str))
seg_top = seg[seg["flight_id"].astype(str).isin(top_ids)]
n_sample_top = min(300_000, len(seg_top))
if n_sample_top > 0:
    seg_top_s = seg_top.sample(n=n_sample_top, random_state=2)
    plt.figure()
    plt.scatter(seg_top_s["mid_lon"], seg_top_s["mid_lat"], s=1, alpha=0.15)
    plt.xlabel("Longitude")
    plt.ylabel("Latitude")
    plt.title(f"Top 2% flights: segment locations, sampled n={n_sample_top:,}")
    plt.show()
else:
    print("No top-2% segments found to plot.")
